In [8]:
!wget -q -O /tmp/pachctl.tar.gz https://github.com/pachyderm/pachyderm/releases/download/v1.13.2/pachctl_1.13.2_linux_amd64.tar.gz 
!tar -xvf /tmp/pachctl.tar.gz -C /tmp 
!cp /tmp/pachctl_1.13.2_linux_amd64/pachctl /opt/conda/bin # Move the binary to somewhere on the PATH

pachctl_1.13.2_linux_amd64/pachctl


In [21]:
!pachctl config update context `./pachctl config get active-context` --pachd-address=pachd:650

In [22]:
!pachctl list repo

NAME CREATED        SIZE (MASTER) DESCRIPTION 
test 10 minutes ago 0B                        


In [23]:
!pachctl create repo test

repo test already exists


In [24]:
!pachctl list repo

NAME CREATED        SIZE (MASTER) DESCRIPTION 
test 10 minutes ago 0B                        
